In [1]:
from keras.models import Sequential, load_model, save_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist, cifar10
from keras.optimizers import SGD
from IPython.display import display
import keras.utils
import keras.backend as K
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import imageio
import json
import os


print(K.tensorflow_backend._get_available_gpus())
print('img data fmt: ', K.image_data_format())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']
img data fmt:  channels_last


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
# convert target labels from scalars to an array of binary values
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# convert img data from ints in range [0,255] to floats in range [0,1] 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

In [3]:
def makeCNN(layer_dict, opt="sgd"):
    cnn = Sequential()
    for key, val in layer_dict.items():
        if (key=='CONV_2D'):
            cnn.add(Conv2D(filters=val[0], 
                           kernel_size=val[1], 
                           input_shape=val[2],
                           activation=val[3]))
        elif (key=='POOL'):
            cnn.add(MaxPooling2D(pool_size=val[0],
                                 strides=val[1]))
        elif (key=='FLATTEN'):
            cnn.add(Flatten())
        elif (key=='DENSE'):
            cnn.add(Dense(units=val[0],
                          activation=val[1]))
    
    cnn.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=opt,
                metrics=['accuracy'])
    return cnn

In [4]:
def writeModelData(model, layer_dict, modelName="new_model", out_dir="..\\..\\store\\csv"):
    
    h5_path = "..\\..\\store\\saved_models\\{}.h5".format(modelName)
    model.save(h5_path)
    
    nLayers = len(layer_dict.keys())
    params = model.count_params()
    hyperParams = 0
    size_kb = 0
    opt="sgd"
    
    model_df = pd.DataFrame(columns=['name', 'layers', 'params', 'hyperParams',
                                     'size_kb', 'filePath', 'optimizer'],
                            data = [[modelName, nLayers, params, hyperParams,
                                    size_kb, h5_path, opt]])
    layer_df = pd.DataFrame(columns=['modelName', 'depth', 'type', 'params', 'filters', 'inputShape',
                                     'outputShape','kernelShape', 'strides'],
                            index=np.arange(nLayers))
    layer_df.loc[:, 'modelName'] = modelName
    depth=0
    for key, val in layer_dict.items():
        layer_df.loc[depth, 'depth'] = depth+1
        layer_df.loc[depth, 'type'] = key
        layer_df.loc[depth, 'params'] = model.layers[depth].count_params()
        layer_df.loc[depth, 'inputShape'] = model.layers[depth].input_shape[1:]
        layer_df.loc[depth, 'outputShape'] = model.layers[depth].output_shape[1:]
        if (key == 'CONV_2D'):
            layer_df.loc[depth, 'filters'] = val[0]
            layer_df.loc[depth, 'kernelShape'] = val[1]
            layer_df.loc[depth, 'strides'] = (1,1)
        elif (key == 'POOL'):
            layer_df.loc[depth, 'filters'] = 0
            layer_df.loc[depth, 'kernelShape'] = val[0]
            layer_df.loc[depth, 'strides'] = val[1]
        depth += 1
    #display(model_df)
    #display(layer_df)
    model_df.to_csv(os.path.join(out_dir, "{}.csv".format(modelName)),
                    header=False)
    layer_df.to_csv(os.path.join(out_dir, "{}_layers.csv".format(modelName)),
                    header=False)
    #model_df.iloc[0,:] = [modelName]
    

In [5]:
input_shape = (28,28,1)

layer_dict1 = {'CONV_2D' : [16, (3,3), input_shape, 'relu'],
               'POOL'    : [(2,2), (2,2)],
               'FLATTEN' : [],
               'DENSE'   : [128, 'relu'],
               'DENSE'   : [10, 'softmax']}

layer_dict2 = {'CONV_2D' : [8, (3,3), input_shape, 'relu'],
               'POOL'    : [(2,2), (2,2)],
               'FLATTEN' : [],
               'DENSE'   : [128, 'relu'],
               'DENSE'   : [10, 'softmax']}

cnn1 = makeCNN(layer_dict1)
cnn2 = makeCNN(layer_dict2)
#cnn1.summary()
#hist1 = trainCNN(cnn1)
#writeModelData(cnn1, layer_dict1, "MNIST_CNN_small")
#writeModelData(cnn2, layer_dict2, "MNIST_CNN_small_2")


In [6]:
def trainCNN(cnn, ep=10, b_size=128, dataset='MNIST', modelName="newModel"):
    hist = cnn.fit(X_train, y_train, batch_size=b_size, epochs=ep)
    return hist

In [7]:
hist1 = trainCNN(cnn1)
hist2 = trainCNN(cnn2)

Epoch 1/10
60000/60000 [==============================] - 3s 50us/step - loss: 1.0756 - acc: 0.7341
Epoch 2/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.3991 - acc: 0.8864
Epoch 3/10
60000/60000 [==============================] - 2s 25us/step - loss: 0.3494 - acc: 0.8990
Epoch 4/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.3277 - acc: 0.9051
Epoch 5/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.3143 - acc: 0.9091
Epoch 6/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.3037 - acc: 0.9118
Epoch 7/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.2950 - acc: 0.9147
Epoch 8/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.2874 - acc: 0.9160
Epoch 9/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.2811 - acc: 0.9193
Epoch 10/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.2744 - acc: 0.9209

In [12]:
hist1_df = pd.DataFrame(hist1.history)
hist2_df = pd.DataFrame(hist2.history)
hist2_df.to_csv("MNIST_CNN_small.csv")
hist1_df.to_csv("MNIST_CNN_small_2.csv")

In [13]:
del cnn1, cnn2